# A Toy Blockchain System
## By Rory Linerud

What follows is my attempt to more or less walk the reader through how a more fleshed out version of my system might be used to perform essential blockchain operations. I begin by sharing here all of my source code for the system, which to date unfortunately remains thoroughly uncommented.

In [1]:
from __future__ import annotations

from cryptography.exceptions import InvalidSignature
from cryptography.hazmat.primitives.asymmetric import padding, rsa
from cryptography.hazmat.primitives import hashes, serialization
from datetime import datetime

import json
import numpy

In [3]:
def int_to_bytes(n: int) -> bytes:
    '''Helpful helper function. Converted ints to bytes'''
    return n.to_bytes((n.bit_length() + 7) // 8, byteorder='big')

In [4]:
class User:
    '''Represents a user of the blockchain.'''

    def __init__(self, public_key: rsa.RSAPublicKey) -> None:
        self.public_key = public_key
        self.modulus = public_key.public_numbers().n
        self.address = numpy.base_repr(self._hash_modulus(), 36)

    def __str__(self) -> str:
        return json.dumps(self, cls=BlockchainEncoder)

    def __repr__(self) -> str:
        return str(self)

    def __eq__(self, other: User) -> bool:
        return self.modulus == other.modulus

    def __hash__(self) -> int:
        return hash(self.modulus)

    def _hash_modulus(self) -> int:
        digest = hashes.Hash(hashes.BLAKE2s(32))
        digest.update(int_to_bytes(self.modulus))
        return int.from_bytes(digest.finalize(), byteorder='big')

In [5]:
class Log:
    '''Represents a single transaction in the blockchain.'''

    def __init__(self, key_path: str, password: str,
                 sender: User, receiver: User, amount: float,
                 item: str | None = None) -> None:
        self.sender = sender
        self.receiver = receiver
        self.amount = amount
        self.item = item
        self.signature = self.sign(key_path, password)
        self.timestamp = str(datetime.now())

    def __str__(self) -> str:
        return json.dumps(self, cls=BlockchainEncoder)

    def __bytes__(self) -> bytes:
        return str(self).encode()

    def __repr__(self) -> str:
        return str(self)

    def sign(self, key_path: str, password: str) -> int:
        '''Sign the transaction to authorize it'''
        with open(key_path, 'rb') as file:
            private_key = serialization.load_pem_private_key(
                file.read(),
                password=password.encode()
            )
            signature = private_key.sign(
                str(self.amount).encode(),
                padding.PSS(
                    mgf=padding.MGF1(hashes.SHA256()),
                    salt_length=padding.PSS.MAX_LENGTH
                ),
                hashes.SHA256()
            )
            return int.from_bytes(signature, byteorder='big')
            #return numpy.base_repr(signature, 36)

    def is_valid(self) -> bool:
        '''Check to see if the sender signed the transaction'''
        try:
            self.sender.public_key.verify(
                int_to_bytes(self.signature),
                str(self.amount).encode(),
                padding.PSS(
                    mgf=padding.MGF1(hashes.SHA256()),
                    salt_length=padding.PSS.MAX_LENGTH
                ),
                hashes.SHA256()
            )
            return True

        except InvalidSignature:
            return False

In [7]:
class Block:
    '''Represents a single block in the blockchain'''

    def __init__(self, logs: list[Log], prev: Block | None = None) -> None:
        self.logs = logs
        if prev:
            self.prev_hash, self.nonce = prev.find_hash()

        else:
            self.prev_hash, self.nonce = None, None

        self.timestamp = str(datetime.now())

    def __str__(self) -> str:
        return json.dumps(self, cls=BlockchainEncoder)

    def __bytes__(self) -> bytes:
        return str(self).encode()

    def __repr__(self) -> str:
        return str(self)

    def find_hash(self) -> tuple[int, int]:
        '''Solve the hash/nonce puzzle'''
        nonce = 0
        digest = self._sha256(nonce)
        while digest % 1_000_000 != 0:
            nonce += 1
            digest = self._sha256(nonce)

        return digest, nonce

    def _sha256(self, nonce: int) -> int:
        digest = hashes.Hash(hashes.SHA256())
        digest.update(bytes(self) + int_to_bytes(nonce))
        return int.from_bytes(digest.finalize(), byteorder='big')

    def is_genesis(self) -> bool:
        return not any(self.nonce, self.prev_hash, self.logs)

In [8]:
class Blockchain:

    def __init__(self) -> None:
        self.blocks = [Block([])]
        self.pending_logs = []
        self.users = {}

    def __len__(self) -> int:
        return len(self.blocks)

    def __str__(self) -> str:
        return json.dumps(self, cls=BlockchainEncoder, indent=4)

    def __repr__(self) -> str:
        return str(self)

    def new_block(self) -> None:
        '''Put all remaining pending transactions into a new block'''
        block = Block(self.pending_logs, self.last_block)
        self.blocks.append(block)
        self.pending_logs = []

    def new_user(
        self,
        password: str,
        private_path: str = 'private.pem',
        public_path: str = 'public.pem',
    ) -> None:
        '''Instantiate a new user on the system'''
        private_key, public_key = self._new_keys()
        user = User(public_key)
        while user.address in self.users:
            private_key, public_key = self._new_keys()
            user = User(public_key)

        self.users[user.address] = user
        self._cache_private_key(private_key, password, private_path)
        self._cache_public_key(public_key, public_path)
        print(f'New wallet created with password "{password}".')
        print(f'Keys stored in "{private_path}" and "{public_path}".')
        print('Do not share your password or private key with anyone!')
        self.new_log(private_path, password, user.address, user.address, 0.0)

    def _new_keys(self) -> tuple[rsa.RSAPrivateKey, rsa.RSAPublicKey]:
        private_key = rsa.generate_private_key(
            public_exponent=65537,
            key_size=2048,
        )
        public_key = private_key.public_key()
        return private_key, public_key

    def _cache_private_key(
        self,
        private_key: rsa.RSAPrivateKey,
        password: str,
        path: str,
    ) -> None:
        algorithm = serialization.BestAvailableEncryption(password.encode())
        private_pem = private_key.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.PKCS8,
            encryption_algorithm=algorithm
        )
        with open(path, 'wb') as file:
            file.write(private_pem)

    def _cache_public_key(
        self,
        public_key: rsa.RSAPublicKey,
        path: str,
    ) -> None:
        public_pem = public_key.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        )
        with open(path, 'wb') as file:
            file.write(public_pem)

    def new_log(
        self,
        key_path: str,
        password: str,
        sender_addr: str,
        receiver_addr: str,
        amount: float,
        item: str | None = None,
    ) -> None:
        sender = self.users[sender_addr]
        receiver = self.users[receiver_addr]
        log = Log(key_path, password, sender, receiver, amount, item)
        self.pending_logs.append(log)

    def is_valid(self) -> bool:
        if len(self) == 1:
            return self.last_block.is_genesis()

        prev_hash, _ = self.penultimate_block.find_hash()
        return prev_hash == self.last_block.prev_hash

    @property
    def penultimate_block(self) -> Block | None:
        if len(self) > 1:
            return self.blocks[-2]

        return None

    @property
    def last_block(self) -> Block:
        return self.blocks[-1]

In [9]:
class BlockchainEncoder(json.JSONEncoder):
    '''Serialize the blockchain into JSON strings'''

    def default(self, obj: Any) -> dict[str, Any]:
        if isinstance(obj, User):
            return {'address': obj.address}

        elif isinstance(obj, Log):
            return {
                'timestamp': obj.timestamp,
                'sender': obj.sender,
                'receiver': obj.receiver,
                'amount': obj.amount,
                'item': obj.item,
                'signature': obj.signature
            }

        elif isinstance(obj, Block):
            return {
                'timestamp': obj.timestamp,
                'nonce': obj.nonce,
                'prevHash': obj.prev_hash,
                'logs': obj.logs
            }

        elif isinstance(obj, Blockchain):
            return {
                'length': len(obj),
                'pendingLogs': obj.pending_logs,
                'blocks': obj.blocks
            }

        return json.JSONEncoder.default(self, obj)

From here, we can begin by instantiating an instance of the top-level Blockchain class.

In [11]:
b = Blockchain()
b

{
    "length": 1,
    "pendingLogs": [],
    "blocks": [
        {
            "timestamp": "2022-04-12 19:05:40.585628",
            "nonce": null,
            "prevHash": null,
            "logs": []
        }
    ]
}

We can see that the blockchain is pretty much completely empty, although it has been given a genesis block to begin the chain. Note that missing fields are accurately represented with a null value in the JSON serialization.

Next, we try instantiating a couple of users to perform trades with one another. Because we are running the blockchain locally, the keys for both of these users will have to be stored on our local machine. Once JSON deserialization is up and running, however, user data should persist between runs so that the local storage of public keys other than your own won't be necessary.

In [12]:
b.new_user(password='banana')

New wallet created with password "banana".
Keys stored in "private.pem" and "public.pem".
Do not share your password or private key with anyone!


In [13]:
b.new_user('orange', 'other_private.pem', 'other_public.pem')

New wallet created with password "orange".
Keys stored in "other_private.pem" and "other_public.pem".
Do not share your password or private key with anyone!


In [14]:
b

{
    "length": 1,
    "pendingLogs": [
        {
            "timestamp": "2022-04-12 19:09:38.709586",
            "sender": {
                "address": "9WODMAQ9MNP0XGBN5VU15XMXM5E84PS7S8O2J126UBWV0YPVS"
            },
            "receiver": {
                "address": "9WODMAQ9MNP0XGBN5VU15XMXM5E84PS7S8O2J126UBWV0YPVS"
            },
            "amount": 0.0,
            "item": null,
            "signature": 202087894111987443997966277155533207611430409141690277838984257426059209097443861452024243152147445364302238428903511701258710693700281750435868389590846575901716794698401813292747351175500623832278851332544267832226875791501289560063556840021403126891011897979097944131678525164053446594263989683928894848027044803207273432049091994969768888994418852538562832627460570580071318155924571782045669292994198644260063398408345980617170552942000526181133404034084654903231474010819766020441052502182135452792488256522479802762283575634771134924054822068344240386720426087645960469632

Printing the blockchain again shows us that two new transactions have been added to the blockchain's pending transactions pool. Each of these correspond to the generation of a new private-public key pair intended for a new user. These transactions are more or less just dummy transactions and don't convey any useful information other than the virtual wallet addresses of the two users. We can go ahead and validate these two transactions (because we know they are valid).

In [15]:
b.new_block()

Doing this takes a few seconds for the blockchain to solve the cryptographic hash puzzle, but printing it out again we see that the two previously pending transactions have since been added to a single block. Also visible is the hash and nonce that satisfied the nonce puzzle, as well as our signature on the block we just created.

In [17]:
b

{
    "length": 2,
    "pendingLogs": [],
    "blocks": [
        {
            "timestamp": "2022-04-12 19:05:40.585628",
            "nonce": null,
            "prevHash": null,
            "logs": []
        },
        {
            "timestamp": "2022-04-12 19:14:00.746925",
            "nonce": 374728,
            "prevHash": 17935035217301825901968154270887183157948782590931860339863917845530353000000,
            "logs": [
                {
                    "timestamp": "2022-04-12 19:09:38.709586",
                    "sender": {
                        "address": "9WODMAQ9MNP0XGBN5VU15XMXM5E84PS7S8O2J126UBWV0YPVS"
                    },
                    "receiver": {
                        "address": "9WODMAQ9MNP0XGBN5VU15XMXM5E84PS7S8O2J126UBWV0YPVS"
                    },
                    "amount": 0.0,
                    "item": null,
                    "signature": 20208789411198744399796627715553320761143040914169027783898425742605920909744386145202424315214744

If another user wanted to, they could confirm that the transactions we just added to the blockchain were actually signed by their senders (us). They could do this like so:

In [20]:
b.last_block.logs[0].is_valid(), b.last_block.logs[1].is_valid()

(True, True)

Next we can try adding a malicious log and see what happens when another user attempts to validate it. First we make a new false transaction, using the virtual addresses present in the chain.

In [22]:
my_addr = '9WODMAQ9MNP0XGBN5VU15XMXM5E84PS7S8O2J126UBWV0YPVS'
other_addr = '2ZG964PCWB6Z7D80ZV0LDOM4OT3WQ9HFUD0RJ3GLGFO3B249W8'

b.new_log('private.pem', 'banana', other_addr, my_addr, 100000.0)

In [24]:
b

{
    "length": 2,
    "pendingLogs": [
        {
            "timestamp": "2022-04-12 19:24:24.574884",
            "sender": {
                "address": "2ZG964PCWB6Z7D80ZV0LDOM4OT3WQ9HFUD0RJ3GLGFO3B249W8"
            },
            "receiver": {
                "address": "9WODMAQ9MNP0XGBN5VU15XMXM5E84PS7S8O2J126UBWV0YPVS"
            },
            "amount": 100000.0,
            "item": null,
            "signature": 174648045557419221934283942969479624997938420051083226976786612788630684235617775732138455844539886541431082881384913666204412079499026694889042617575790170665828187287628080058129931923347167805079273191789336681924897160550310778248110819808086019008834871152840682185921768057109164924668820961288127132992986016767948123778821376695533393566861209117743527027279696661008165188487241261294828858157960742338418294873816967120498546762424091104134208852624431173876924027352135557556543044967919053256727000580611254905557068802551945003854172662329593772574509582448563

We see that the log has been added without any error being raised. Oh no! Luckily users can still check the validity of transactions manually. In this case it is quite straightforward, as the transaction's signature will not match the sender's key.

In [25]:
b.pending_logs[0].is_valid()

False

In response, we can simply remove the log from the pending logs pool and go on with our day.

In [26]:
b.pending_logs.clear()
b

{
    "length": 2,
    "pendingLogs": [],
    "blocks": [
        {
            "timestamp": "2022-04-12 19:05:40.585628",
            "nonce": null,
            "prevHash": null,
            "logs": []
        },
        {
            "timestamp": "2022-04-12 19:14:00.746925",
            "nonce": 374728,
            "prevHash": 17935035217301825901968154270887183157948782590931860339863917845530353000000,
            "logs": [
                {
                    "timestamp": "2022-04-12 19:09:38.709586",
                    "sender": {
                        "address": "9WODMAQ9MNP0XGBN5VU15XMXM5E84PS7S8O2J126UBWV0YPVS"
                    },
                    "receiver": {
                        "address": "9WODMAQ9MNP0XGBN5VU15XMXM5E84PS7S8O2J126UBWV0YPVS"
                    },
                    "amount": 0.0,
                    "item": null,
                    "signature": 20208789411198744399796627715553320761143040914169027783898425742605920909744386145202424315214744

Finally, we show that basic NFT functionality is included with this blockchain too. All we really need to do is add an IPFS hash of whatever object we want to trade as an argument to the new_log method call. To generate this hash, we do need IPFS installed on our local system.

In [27]:
%%bash

ipfs help

USAGE
  ipfs  - Global p2p merkle-dag filesystem.

SYNOPSIS
  ipfs [--config=<config> | -c] [--debug | -D] [--help] [-h] [--api=<api>] [--offline] [--cid-base=<base>] [--upgrade-cidv0-in-output] [--encoding=<encoding> | --enc] [--timeout=<timeout>] <command> ...

OPTIONS

  -c, --config               string - Path to the configuration file to use.
  -D, --debug                bool   - Operate in debug mode.
  --help                     bool   - Show the full command help text.
  -h                         bool   - Show a short version of the command help
                                      text.
  -L, --local                bool   - Run the command locally, instead of using
                                      the daemon. DEPRECATED: use --offline.
  --offline                  bool   - Run the command offline.
  --api                      string - Use a specific API instance (defaults to
                                      /ip4/127.0.0.1/tcp/5001).
  --cid-base                 str

We can upload files to IPFS pretty easily. Doing so will return the IPFS hash of the file back to you so you may retrieve it later on. To download files back from the distributed IPFS system, the daemon will probably need to be activated first.

In [28]:
%%bash

ipfs add kaboom.png

added QmdNAQU5wi61qBW2inREccbhoQWUd3YfuheXunUSmLZpmS kaboom.png


 87.89 KiB / 87.89 KiB  100.00%

In [29]:
ipfs_hash = 'QmdNAQU5wi61qBW2inREccbhoQWUd3YfuheXunUSmLZpmS'

In [30]:
%%bash

ipfs cat QmdNAQU5wi61qBW2inREccbhoQWUd3YfuheXunUSmLZpmS > kaboom_copy.png

Once the IPFS hash of the file is ready, you can simply add as an argument to the new_log method call. This hash will persist on the blockchain as long as the whole blockchain persists in memory, and cannot be deleted or moved to another user without invalidating the entirety of the blockchain.

In [32]:
b.new_log('private.pem', 'banana', my_addr, other_addr, 0.0, ipfs_hash)

In [33]:
b

{
    "length": 2,
    "pendingLogs": [
        {
            "timestamp": "2022-04-12 19:40:05.867113",
            "sender": {
                "address": "9WODMAQ9MNP0XGBN5VU15XMXM5E84PS7S8O2J126UBWV0YPVS"
            },
            "receiver": {
                "address": "2ZG964PCWB6Z7D80ZV0LDOM4OT3WQ9HFUD0RJ3GLGFO3B249W8"
            },
            "amount": 0.0,
            "item": "QmdNAQU5wi61qBW2inREccbhoQWUd3YfuheXunUSmLZpmS",
            "signature": 759225982975922726902593134734669918243224557097170479434398129648072802375101818720492574132145642336784734233802957518468581823768257415729121582058048523894541963169105098831156694561384514497791863176154812982429979558921952995499550818846613566174169171179282260874578634760453358409784636535078438840973374400560449386002386077394929171238965975726933089013536071366457826624017891120748062954004097889436420751179253653580684823470562284642295541089707194261064224490820536286495098393542338355478743586450936918692716415331245

And voila! An NFT has successfully been transferred from one user to another for free.